In [7]:
spark_version = "3.5"
scala_version = "2.12"
iceberg_version = "1.7.0"

from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, rand, floor, expr

catalog_name = "iceberg"
warehouse_path = "./icehouse"

spark = SparkSession.builder \
    .appName("local_iceberg_example") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version}") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/matthewmartin/dream_machine/substack/articles/2025.02.07-Why_partitions_matter/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthewmartin/.ivy2/cache
The jars for the packages stored in: /Users/matthewmartin/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-917502df-1ce2-4813-ba95-22c47fb02128;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 in central
:: resolution report :: resolve 39ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :

In [8]:
namespace = "test_ns"

In [1]:
import data_generator as dg

In [2]:
row_cnt = 10_000_000
row_cnt = 100_000

In [3]:
df = dg.generate_dummy_order_data(row_cnt,1)

In [4]:
dfo = dg.generate_dummy_order_details(df, 5)

In [ ]:

dg.save_csv_files(df, dfo, ".", "v1")

Saving header file to ./raw_data/ord_hdr/order_header_v1.csv
Saving detail file to ./raw_data/ord_dtl/order_detail_v1.csv


In [9]:
dfh = spark.read.csv("./raw_data/ord_hdr/*", header=True)
dfd = spark.read.csv("./raw_data/ord_dtl/*", header=True)

In [10]:
dfh.head(5)

[Row(order_date='2023-01-01', order_id='ORD-00001', customer_id='CUST-0101', order_status='Shipped', order_total='213.24'),
 Row(order_date='2023-01-01', order_id='ORD-00002', customer_id='CUST-0345', order_status='Delivered', order_total='982.84'),
 Row(order_date='2023-01-01', order_id='ORD-00003', customer_id='CUST-0436', order_status='Cancelled', order_total='1020.06'),
 Row(order_date='2023-01-01', order_id='ORD-00004', customer_id='CUST-0321', order_status='Pending', order_total='1175.73'),
 Row(order_date='2023-01-01', order_id='ORD-00005', customer_id='CUST-0435', order_status='Shipped', order_total='1184.62')]

In [ ]:
table_name = "ord_hdr_no_partition"
dfh.writeTo(f"{catalog_name}.{namespace}.{table_name}") \
    .using("iceberg") \
    .createOrReplace()

In [ ]:
table_name = "ord_dtl_no_partition"
dfd.writeTo(f"{catalog_name}.{namespace}.{table_name}") \
    .using("iceberg") \
    .createOrReplace()

In [13]:
table_name = "ord_hdr_partitioned"
dfh.createOrReplaceTempView("hdr")

spark.sql(f"""
        CREATE OR REPLACE TABLE {catalog_name}.{namespace}.{table_name}
        USING ICEBERG
        PARTITIONED BY (order_date)
        AS SELECT *
        FROM hdr
          """)


25/02/05 05:58:21 WARN HadoopTableOperations: Error reading version hint file icehouse/test_ns/ord_hdr_partitioned/metadata/version-hint.text
java.io.FileNotFoundException: File icehouse/test_ns/ord_hdr_partitioned/metadata/version-hint.text does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hadoop.HadoopTableOperations.findVersion(HadoopTableOperations.java:317)
	at org.apache.iceberg.hadoop.HadoopT

DataFrame[]

generate 100m row table x 2 in iceberg no partitions
generate same dataset with partitions
test perf
done

In [ ]:
df_agg.writeTo(f"{catalog_name}.{namespace}.{table_name}") \
    .using("iceberg") \
    .createOrReplace()